### The purpose of this notebook is to keep track of all the different avenues the Data Quality team explores in attempting to understand the capacity & scalability, and efficiency & performance of our freelancers

In [1]:
## For an AR, generate a list of the account numbers that are on a freelancer's dashboard
# This script can be used during an Account Review to ideintify which of the account numbers in the account
# are currently on a freelancer's dashboard. Anybody working on the AR would be able to ping a freelancer to
# prioritize these accounts or simply remove these accounts from the AR at the beginning, 
# eliminating the time and effort required of the individual to update the account themselves

import selenium
import csv
import time
from selenium import webdriver

## Downloading freelancer dashboards
# Using Chrome to access web
driver = webdriver.Chrome()

# Open the website
driver.get('https://www.wegowise.com/login')

# Select id box
time.sleep(2)
id_box = driver.find_element_by_id('username')
id_box.send_keys('kojiro')

time.sleep(1)

# Find password box
pass_box = driver.find_element_by_id('password')

# Send password
pass_box.send_keys('Koj!rom1yagawa')

# Find login button
time.sleep(1)
login_button = driver.find_element_by_name('go_raquo__button')

# Click login
login_button.click()

freelancer_list = ["aladao", "cching", "crollins","cjones","eoracion","sosmanollaj",
                   "mjuma", "rasentista", "lmaina", "abernardo","tmaniulit", "rsantos", "jsmolinski", "rfoster"]

k = 0 

for i in freelancer_list:
    j = freelancer_list[k]
    #Search freelancer
    time.sleep(3)
    search_box = driver.find_element_by_id("query")
    search_box.send_keys(j)
    search_button = driver.find_element_by_name("search_button")
    search_button.click()

    # Click on freelancer name
    time.sleep(1)
    freelancer = driver.find_element_by_link_text(j)
    freelancer.click()

    # Click on download button
    time.sleep(1)
    dl_button = driver.find_element_by_class_name('icon-disk')
    dl_button.click()
    
    # Return to search page
    time.sleep(1)
    driver.back()
    time.sleep(1)
    driver.back()
    k = k+1

In [4]:
## Joining all dashboards
import pandas as pd

aladao = pd.read_csv('/Users/kojiroso/Downloads/data_entry_aladao.csv')
aladao = aladao.assign(freelancer_id='aladao')

cching = pd.read_csv("/Users/kojiroso/Downloads/data_entry_cching.csv")
cching = cching.assign(freelancer_id='cching')

crollins = pd.read_csv("/Users/kojiroso/Downloads/data_entry_crollins.csv")
crollins = crollins.assign(freelancer_id='crollins')

# dfrancis = pd.read_csv("/Users/kojiroso/Downloads/data_entry_dfrancis.csv")
# dfrancis = dfrancis.assign(freelancer_id='dfrancis')

cjones = pd.read_csv("/Users/kojiroso/Downloads/data_entry_cjones.csv")
cjones = cjones.assign(freelancer_id='cjones')

eoracion = pd.read_csv("/Users/kojiroso/Downloads/data_entry_eoracion.csv")
eoracion = eoracion.assign(freelancer_id='eoracion')

sosmanollaj = pd.read_csv("/Users/kojiroso/Downloads/data_entry_sosmanollaj.csv")
sosmanollaj = sosmanollaj.assign(freelancer_id='sosmanollaj')

mjuma = pd.read_csv("/Users/kojiroso/Downloads/data_entry_mjuma.csv")
mjuma = mjuma.assign(freelancer_id='mjuma')

rasentista = pd.read_csv("/Users/kojiroso/Downloads/data_entry_rasentista.csv")
rasentista = rasentista.assign(freelancer_id='rasentista')

lmaina = pd.read_csv("/Users/kojiroso/Downloads/data_entry_lmaina.csv")
lmaina = lmaina.assign(freelancer_id ='lmaina')

abernardo = pd.read_csv("/Users/kojiroso/Downloads/data_entry_abernardo.csv")
abernardo = abernardo.assign(freelancer_id = 'abernardo')

tmaniulit = pd.read_csv("/Users/kojiroso/Downloads/data_entry_tmaniulit.csv")
tmaniulit = tmaniulit.assign(freelancer_id = 'tmaniulit')

rsantos = pd.read_csv("/Users/kojiroso/Downloads/data_entry_rsantos.csv")
rsantos = rsantos.assign(freelancer_id = 'rsantos')

jsmolinski = pd.read_csv("/Users/kojiroso/Downloads/data_entry_jsmolinski.csv")
jsmolinski = jsmolinski.assign(freelancer_id = 'jsmolinski')

rfoster = pd.read_csv("/Users/kojiroso/Downloads/data_entry_rfoster.csv")
rfoster = rfoster.assign(freelancer_id = 'rfoster')

freelancers = pd.concat([aladao, cching, crollins, cjones, eoracion, sosmanollaj, 
           mjuma, rasentista, lmaina, abernardo, tmaniulit, rsantos, jsmolinski, rfoster])

In [82]:
## Number of accounts on freelancers' dashboards that are 'out-of-date' i.e. Est Next Datapoint is in the past
## This includes account numbers where Est Next Datapoint is blank

import numpy as np
from datetime import datetime, timedelta
today = (datetime.today()-timedelta(10)).strftime('%Y-%m-%d')

## Total number of accounts by freelancer

# Filter data so we're only looking at accounts with an Est Next Datapoint in the past/ with blank Est Next Datapoints
fl_filter = freelancers[(freelancers['Est Next Datapoint'] <= today)| (freelancers['Est Next Datapoint'].isnull())]
# Count number of out-of-date meters by freelancer
old_meter_count = fl_filter.groupby(['freelancer_id','Utility Company'])['Account Number'].count()
old_meter_count = old_meter_count.groupby(level=0, group_keys=False).nlargest(5)
old_meter_count = old_meter_count.to_frame().head(30)


# fl_filter.sort_values(['freelancer_id',''])


# import pandas as pd
# import numpy as np
# def is_dataframe(x):
#     if isinstance(x, pd.DataFrame):
#         print(1)
#     else:
#         raise Exception('wrong type')
        
# import pandas as pd
# import numpy as np
# def is_something_else(l):
#     if isinstance(l,(pd.core.series.Series)):
#         print(5)
#     else:
#         raise Exception('wrong type')
        
# is_something_else(old_meter_count)



In [83]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table

app = dash.Dash(__name__)

app.layout = dash_table.DataTable(
    id='table',
    columns=[{"name":i, "id":i} for i in old_meter_count.columns],
    data=old_meter_count.to_dict('records'),
)

if __name__ == '__main__':
    app.run_server(debug=True)

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 437-656-934
Debugger PIN: 437-656-934
Debugger PIN: 437-656-934
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

/Users/kojiroso/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3304: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.



In [ ]:
# Count number of out-of-date meters by freelancer
old_meter_count = fl_filter.groupby(['freelancer_id','Utility Company'])['Account Number'].count()
# old_meter_count = old_meter_count.to_frame()
# old_meter_count = old_meter_count.rename(columns={'Account Number':'Number of Meters'})
old_meter_count = old_meter_count.nlargest(5)
old_meter_count.head(20)

# Count number of out-of-date meters by freelancer
old_meter_count = fl_filter.groupby(['freelancer_id','Utility Company'])['Account Number'].count()

# .reset_index(name='old_meters')

# Sort by freelancer with the most out-of-date meters
old_meter_count = old_meter_count.freelancer_id.sort_values()

# , 'old_meters'

# Group by and largest 3 rows
old_meter_count = old_meter_count.groupby(['freelancer_id','Utility Company']).nlargest(5)

# ['old_meters']

# Export as excel file
old_meter_count.to_excel("old_meter_count.xlsx")

In [5]:
## Total number of accounts by freelancer, by month

# Number of accounts by freelancer by date 
fl_filter.groupby(['freelancer_id','Est Next Datapoint'])['Account Number'].count()

# Convert dates to datetime data type
fl_filter['Est Next Datapoint'] = pd.to_datetime(fl_filter['Est Next Datapoint'])

# Convert dates to year-month format
fl_filter['month_year'] = fl_filter['Est Next Datapoint'].dt.to_period('M')

# Total number of out-of-date meters by freelancer by month
old_meter_count_by_month = fl_filter.groupby(['freelancer_id','month_year'])['Account Number'].count().reset_index(name='no_of_meters')

old_meter_count_by_month = old_meter_count_by_month.pivot(index='freelancer_id', columns = 'month_year')['no_of_meters']

# Export as excel file
old_meter_count_by_month.to_excel("old_meter_count_by_month.xlsx")

/Users/kojiroso/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/kojiroso/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


In [5]:
## Joining all dashboards
import pandas as pd

aladao = pd.read_csv('/Users/kojiroso/Downloads/data_entry_aladao.csv')
aladao = aladao.assign(freelancer_id='aladao')

cching = pd.read_csv("/Users/kojiroso/Downloads/data_entry_cching.csv")
cching = cching.assign(freelancer_id='cching')

crollins = pd.read_csv("/Users/kojiroso/Downloads/data_entry_crollins.csv")
crollins = crollins.assign(freelancer_id='crollins')

# dfrancis = pd.read_csv("/Users/kojiroso/Downloads/data_entry_dfrancis.csv")
# dfrancis = dfrancis.assign(freelancer_id='dfrancis')

cjones = pd.read_csv("/Users/kojiroso/Downloads/data_entry_cjones.csv")
cjones = cjones.assign(freelancer_id='cjones')

eoracion = pd.read_csv("/Users/kojiroso/Downloads/data_entry_eoracion.csv")
eoracion = eoracion.assign(freelancer_id='eoracion')

sosmanollaj = pd.read_csv("/Users/kojiroso/Downloads/data_entry_sosmanollaj.csv")
sosmanollaj = sosmanollaj.assign(freelancer_id='sosmanollaj')

mjuma = pd.read_csv("/Users/kojiroso/Downloads/data_entry_mjuma.csv")
mjuma = mjuma.assign(freelancer_id='mjuma')

rasentista = pd.read_csv("/Users/kojiroso/Downloads/data_entry_rasentista.csv")
rasentista = rasentista.assign(freelancer_id='rasentista')

lmaina = pd.read_csv("/Users/kojiroso/Downloads/data_entry_lmaina.csv")
lmaina = lmaina.assign(freelancer_id ='lmaina')

abernardo = pd.read_csv("/Users/kojiroso/Downloads/data_entry_abernardo.csv")
abernardo = abernardo.assign(freelancer_id = 'abernardo')

tmaniulit = pd.read_csv("/Users/kojiroso/Downloads/data_entry_tmaniulit.csv")
tmaniulit = tmaniulit.assign(freelancer_id = 'tmaniulit')

rsantos = pd.read_csv("/Users/kojiroso/Downloads/data_entry_rsantos.csv")
rsantos = rsantos.assign(freelancer_id = 'rsantos')

jsmolinski = pd.read_csv("/Users/kojiroso/Downloads/data_entry_jsmolinski.csv")
jsmolinski = jsmolinski.assign(freelancer_id = 'jsmolinski')

rfoster = pd.read_csv("/Users/kojiroso/Downloads/data_entry_rfoster.csv")
rfoster = rfoster.assign(freelancer_id = 'rfoster')

freelancers = pd.concat([aladao, cching, crollins, cjones, eoracion, sosmanollaj, 
           mjuma, rasentista, lmaina, abernardo, tmaniulit, rsantos, jsmolinski, rfoster])

## Number of accounts on freelancers' dashboards that are 'out-of-date' i.e. Est Next Datapoint is in the past
## This includes account numbers where Est Next Datapoint is blank

import numpy as np
from datetime import datetime, timedelta
today = (datetime.today()-timedelta(10)).strftime('%Y-%m-%d')

## Total number of accounts by freelancer

# Filter data so we're only looking at accounts with an Est Next Datapoint in the past/ with blank Est Next Datapoints
fl_filter = freelancers[(freelancers['Est Next Datapoint'] <= today)| (freelancers['Est Next Datapoint'].isnull())]
# Count number of out-of-date meters by freelancer
old_meter_count = fl_filter.groupby(['freelancer_id','Utility Company'])['Account Number'].count()
old_meter_count = old_meter_count.groupby(level=0, group_keys=False).nlargest(5)
old_meter_count = old_meter_count.to_frame()
old_meter_count = old_meter_count.reset_index(level=['freelancer_id','Utility Company'])

# import dash
# import dash_core_components as dcc
# import dash_html_components as html
# import dash_table

# app = dash.Dash(__name__)

# app.layout = dash_table.DataTable(
#     id='table',
#     columns=[{"name":i, "id":i} for i in old_meter_count.columns],
#     data=old_meter_count.to_dict('records'),
# )

# if __name__ == '__main__':
#     app.run_server(debug=True)

